# USE API'S TO UPDATE MYSQL DATABASE WITH SCHEDULER USING PYTHON

# Change Passwords.. Back

In [10]:
!pip install mysql-connector-python
import mysql.connector
#https://realpython.com/python-mysql/

# CONNECTING THE EASIER WAY

In [40]:

mydb = mysql.connector.connect(
  host="localhost",
  user="blake",
  password="SW.."
)

print(mydb)

In [41]:
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE worldinformation")

In [42]:
# SHOW THE DATABASE JUST CREATED
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('cat',)
('covid19',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('world',)
('worldinformation',)


In [135]:
# Connecting to the database just created 
mydb = mysql.connector.connect(
  host="localhost",
  user="blake",
  password="SW..",
  database="worldinformation",
  connect_timeout=1000 ,
)

# Creating table to hold Covid-19 Data (DONT NEED THIS... SQL ALCHEMY INJECTS DIRECTLY)

In [48]:
# CREATE TABLE FOR COVID 19 GITHUB DATA (USA STATE LEVEL) FROM https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
# MAKE THE STATE THE PRIMARY KEY... (COMEBACK AND REEVALUATE IF THIS IS APPROPRIATE OR AUTOINCREMENT)

#mycursor = mydb.cursor()
#mycursor.execute("CREATE TABLE covids (date DATE ,state VARCHAR(100) NOT NULL ,fips INT,cases INT,deaths INT,primary key (state))")


In [49]:
# Check to see the table and if it exists
##mycursor.execute("SHOW TABLES")

#for x in mycursor:
 # print(x)

('covids',)


# Download Covid data and upload to MYSQL through INSERT
# STATE LEVEL INFORMATION
# COVID

In [58]:
!pip install pymysql

In [69]:
#KZO00183347L
import pandas as pd
import numpy as np
from pandas.io import sql
import pymysql
import sqlalchemy
user = 'blake'
passw = 'SW..'
host =  'localhost'
port = 3306
database = 'worldinformation'



In [109]:
statecovid = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv")
statecovid['date'] = pd.to_datetime(statecovid['date'])
statecovid['state'] = statecovid['state'].astype(str)
statecovid = pd.DataFrame(statecovid)
statecovid.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [110]:
database_username = 'blake'
database_password = 'SW..'
database_ip       = 'localhost'
database_name     = 'worldinformation'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
statecovid.to_sql(con=database_connection, name='usacovid', if_exists='replace')

# DOWNLOAD USA STATE MOBILITY

In [79]:
usa_mobility = pd.read_csv("https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_US.csv")
usa_mobility['date'] = pd.to_datetime(usa_mobility['date'])
usa_mobility['retail and recreation'] = usa_mobility['retail and recreation'].fillna(0)
usa_mobility['grocery and pharmacy'] = usa_mobility['grocery and pharmacy'].fillna(0)
usa_mobility['parks'] = usa_mobility['parks'].fillna(0)
usa_mobility['transit stations'] = usa_mobility['transit stations'].fillna(0)
usa_mobility['workplaces'] = usa_mobility['workplaces'].fillna(0)
usa_mobility['residential'] = usa_mobility['residential'].fillna(0)
usa_mobility1 = usa_mobility.groupby(['date','state'])[['retail and recreation', 'grocery and pharmacy','parks','transit stations','workplaces','residential']].mean().reset_index()
usa_mobility1['retail and recreation'] = round(usa_mobility1['retail and recreation'],2)
usa_mobility1['grocery and pharmacy'] = round(usa_mobility1['grocery and pharmacy'],2)
usa_mobility1['parks'] = round(usa_mobility1['parks'],2)
usa_mobility1['transit stations'] = round(usa_mobility1['transit stations'],2)
usa_mobility1['workplaces'] = round(usa_mobility1['workplaces'],2)
usa_mobility1['residential'] = round(usa_mobility1['residential'],2)
#usa_mobility1.index = pd.to_datetime(usa_mobility['date'])
usa_mobility1['date'] = pd.to_datetime(usa_mobility1['date'])
us_mobility1 = pd.DataFrame(usa_mobility1)
usa_mobility1.head()
#usa_mobility1.isnull().sum()

,date,state,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
0,2020-02-15,Alabama,4.46,0.22,10.36,1.81,0.60,-0.28
1,2020-02-15,Alaska,6.38,0.62,3.12,1.12,-0.38,-0.12
2,2020-02-15,Arizona,4.75,3.69,16.31,8.38,-1.25,-0.19
3,2020-02-15,Arkansas,1.23,1.00,1.68,-1.17,0.48,0.02
4,2020-02-15,California,6.34,4.50,26.34,3.73,-1.70,-0.46


In [80]:
# SEND MOVILIY TO DATBASE
usa_mobility1.to_sql(con=database_connection, name='usamobility', if_exists='replace')

# COUNTRY LEVEL COVID

In [92]:
url_confirmed3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
confirmed3 = pd.read_csv(url_confirmed3)
deaths3 = pd.read_csv(url_deaths3)
recovered3 = pd.read_csv(url_recovered3)
# Unpivot data frames
date13 = confirmed3.columns[4:]
total_confirmed3 = confirmed3.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=date13, var_name='date', value_name='confirmed')
date23 = deaths3.columns[4:]
total_deaths3 = deaths3.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=date23, var_name='date', value_name='death')
date33 = recovered3.columns[4:]
total_recovered3 = recovered3.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=date33, var_name='date', value_name='recovered')
# Merging data frames
covid_data3 = total_confirmed3.merge(right=total_deaths3, how='left', on=['Province/State', 'Country/Region', 'date', 'Lat', 'Long'])
covid_data3 = covid_data3.merge(right=total_recovered3, how='left', on=['Province/State', 'Country/Region', 'date', 'Lat', 'Long'])
# Converting date column from string to proper date format
#covid_data['date'] = pd.to_datetime(covid_data['date'])
# Check how many missing value naN
#covid_data.isna().sum()
# Replace naN with 0
covid_data3['recovered'] = covid_data3['recovered'].fillna(0)
# Calculate new column
covid_data3['active'] = covid_data3['confirmed'] - covid_data3['death'] - covid_data3['recovered']
# By date for all countries (NOT FOR GRAPH)
#This is also for last updated caluclation!
covid_data23 = covid_data3
covid_data23['date'] = pd.to_datetime(covid_data23['date'])
#covid_data23.isnull().sum()
#covid_data23['Province/State'] = covid_data23['Province/State'].fillna('-')
#covid_data23['Lat'] = covid_data23['Lat'].fillna('-')
#covid_data23['Long'] = covid_data23['Long'].fillna('-')
#covid_data23.isnull().sum()
covid_data23 = pd.DataFrame(covid_data23)

In [100]:
list(covid_data23.columns)

['Province/State',
 'Country/Region',
 'Lat',
 'Long',
 'date',
 'confirmed',
 'death',
 'recovered',
 'active']

In [ ]:
# DO THIS IN MYSQL SERVER... TO ALLOW FOR BIG DATA DUMPS 
set global max_allowed_packet=67108864;


In [108]:
# IF I WANT TO CREATE THE TABLE TO HOLD THE WORLD COVID BEFORE I INSERT IT INTO DATA BASE DO THIS FIRST... OTHERWISE ALTER IN MYSQL..
#mycursor = mydb.cursor()
# CANT HAVE / IN COLUMN NAME FOR SOME REASON...
#mycursor.execute("CREATE TABLE worldcovid2 (Province/State VARCHAR(100) ,Country/Region VARCHAR(100),Lat DECIMAL(2) ,Long DECIMAL(2),date DATE,confirmed INT,death INT,recovered INT,active INT)")


In [128]:
database_username = 'blake'
database_password = 'SW..'
database_ip       = 'localhost'
database_name     = 'worldinformation'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
covid_data23.to_sql(con=database_connection, name='worldcovid', if_exists='replace')

# COUNTRY LEVEL MOBILITY

In [114]:
mobility_countries = pd.read_csv("https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_countries.csv")
#mobility_countries['date'] = pd.to_datetime(mobility_countries['date'])

mobility_countries['retail and recreation'] = mobility_countries['retail and recreation'].fillna(0)
mobility_countries['grocery and pharmacy'] = mobility_countries['grocery and pharmacy'].fillna(0)
mobility_countries['parks'] = mobility_countries['parks'].fillna(0)
mobility_countries['transit stations'] = mobility_countries['transit stations'].fillna(0)
mobility_countries['workplaces'] = mobility_countries['workplaces'].fillna(0)
mobility_countries['residential'] = mobility_countries['residential'].fillna(0)

mobility_countries1 = mobility_countries.groupby(['date','country'])[['retail and recreation', 'grocery and pharmacy','parks','transit stations','workplaces','residential']].mean().reset_index()
mobility_countries1['retail and recreation'] = round(mobility_countries1['retail and recreation'],2)
mobility_countries1['grocery and pharmacy'] = round(mobility_countries1['grocery and pharmacy'],2)
mobility_countries1['parks'] = round(mobility_countries1['parks'],2)
mobility_countries1['transit stations'] = round(mobility_countries1['transit stations'],2)
mobility_countries1['workplaces'] = round(mobility_countries1['workplaces'],2)
mobility_countries1['residential'] = round(mobility_countries1['residential'],2)
#usa_mobility1.index = pd.to_datetime(usa_mobility['date'])
mobility_countries1['date'] = pd.to_datetime(mobility_countries1['date'])
#mobility_countries1.set_index('date', inplace=True)
mobility_countries1 = pd.DataFrame(mobility_countries1)
mobility_countries1

,date,country,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
0,2020-02-15,Afghanistan,-9.00,-10.00,0.00,-2.00,-28.00,5.00
1,2020-02-15,Angola,1.00,-1.33,3.33,1.00,2.00,0.67
2,2020-02-15,Antigua and Barbuda,-3.67,-0.33,-3.33,1.33,-0.33,1.00
3,2020-02-15,Argentina,0.08,-2.12,-6.32,-5.36,-0.92,1.28
4,2020-02-15,Aruba,2.00,3.00,12.00,26.00,-2.00,-2.00
...,...,...,...,...,...,...,...,...
46348,2021-01-24,Venezuela,-40.00,-12.00,-34.00,-48.00,-12.00,11.00
46349,2021-01-24,Vietnam,-12.33,1.27,-17.19,-14.19,-15.14,-8.95
46350,2021-01-24,Yemen,24.00,40.00,31.00,7.00,3.00,2.00
46351,2021-01-24,Zambia,-14.00,-6.00,4.00,-28.00,-24.00,13.00


In [115]:
mobility_countries1.to_sql(con=database_connection, name='worldmobility', if_exists='replace')

# USA County Level Covid Data

In [123]:
county_covid = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")
county_covid['date'] = pd.to_datetime(county_covid['date'])
county_covid['deaths'] = county_covid['deaths'].fillna(0)
county_covid['fips'] = county_covid['fips'].fillna(0)
county_covid = pd.DataFrame(county_covid)
county_covid.head()

# GO TO MYSQL AND DO THIS 

#set global max_allowed_packet=67108864;
#SET session wait_timeout=7000;

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [146]:
county_covid.to_sql(con=database_connection, name='usacountycovid', if_exists='replace',chunksize=100000, method='multi')

# USA County Level Mobility Data

In [143]:
# County level mobility data 
usa_county_mobility= pd.read_csv("https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_US.csv")
usa_county_mobility['date'] = pd.to_datetime(usa_county_mobility['date'])
usa_county_mobility['retail and recreation'] = usa_county_mobility['retail and recreation'].fillna(0)
usa_county_mobility['grocery and pharmacy'] = usa_county_mobility['grocery and pharmacy'].fillna(0)
usa_county_mobility['parks'] = usa_county_mobility['parks'].fillna(0)
usa_county_mobility['transit stations'] = usa_county_mobility['transit stations'].fillna(0)
usa_county_mobility['workplaces'] = usa_county_mobility['workplaces'].fillna(0)
usa_county_mobility['residential'] = usa_county_mobility['residential'].fillna(0)
usa_county_mobility = pd.DataFrame(usa_county_mobility)
usa_county_mobility.head()
usa_county_mobility3 = usa_county_mobility.head()
usa_county_mobility.shape


(874065, 9)

In [145]:
usa_county_mobility.to_sql(con=database_connection, name='usacountymobility', if_exists='replace',chunksize=10000, method='multi')